#Agentic RAG with ApertureDB and Hugging Face SmolAgents


This implementation highlights the Agentic RAG implementation using ApertureDB data store, which is a graph-based multimodal database. Hugging Face SmolAgents will be employed for implementing a multi-agent LLM workflow.\

# 1. Prepping the data

For this implementation, we’ll be using an Arxiv structured complex dataset. Which will be first pre-processed into embeddings so that they can be stored in the ApertureDB vector database. The large dataset is divided into small chunks of data, out of which vector embeddings are generated using a model like sentence-transformer/all-MiniLM-L6-v2 from Hugging Face. These vector embeddings are then stored in apertureDB.


In [ ]:
# arxiv complex structured data is used as dataset

In [ ]:
import arxiv
from sentence_transformers import SentenceTransformer
from typing import List
import numpy as np

def get_arxiv_papers(query: str, max_results: int = 10) -> List[str]:
    """Fetch papers from arXiv and return their text content"""
    client = arxiv.Client()
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )

    papers = []
    for result in client.results(search):
        papers.append(f"Title: {result.title}\nAbstract: {result.summary}")

    return papers

def chunk_text(text: str, chunk_size: int = 512) -> List[str]:
    """Split text into chunks of specified size"""
    words = text.split()
    chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

# Fetch and prepare data
arxiv_query = "graph embeddings"
papers = get_arxiv_papers(arxiv_query)
chunks = []
for paper in papers:
    chunks.extend(chunk_text(paper))

# Generate embeddings
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = model.encode(chunks, show_progress_bar=True)

print(f"Generated {len(embeddings)} embeddings for {len(chunks)} chunks")


# 2. Setting Up ApertureDB

In this implementation, ApertureDB is used as a vector store that offers high performance and speedy retrievals, also supporting multimodal data. The vector based stores help in maintaining context as vectors while generating responses. ApertureDB also integrates seamlessly with AI pipelines like LangChain and Hugging Face. This is how you store vector embeddings in ApertureDB.

In [ ]:
import aperturedb
from aperturedb.Connector import Connector
import time
class ApertureDBVectorStore:
    def __init__(self, host: str = "localhost", port: int = 8000):
        self.client = Connector({"host": host, "port": port})
    def create_schema(self):
        """Create necessary schema in ApertureDB"""
        schema = [ {
                "CreateProperty": {
                    "name": "type",
                    "type": "string"
                }
            },
            {
                "CreateProperty": {
                    "name": "text",
                    "type": "string"
                }
            },
            {
                "CreateVectorIndex": {
                    "name": "default_embedding",
                    "dimensions": 384,
                    "metric": "cosine"
                }
            }
        ]
        response, _ = self.client.query(schema)
        return response
    def store_embeddings(self, chunks: List[str], embeddings: np.ndarray):
        """Store text chunks with their embeddings in ApertureDB"""
        records = []
        for chunk, embedding in zip(chunks, embeddings):
            records.append([
                {
                    "AddObject": {
                        "properties": {
                            "type": "document",
                            "text": chunk
                        },
                        "embedding": {
                            "vector": embedding.tolist(),
                            "name": "default_embedding"
                        }
                    }
                }
            ])
        batch_size = 50
        for i in range(0, len(records), batch_size):
            batch = records[i:i+batch_size]
            response, blobs = self.client.query(batch)
            print(f"Inserted batch {i//batch_size + 1}, response: {response}")
            time.sleep(0.1)
        return True
# Initialize and populate ApertureDB
aperture_db = ApertureDBVectorStore()
aperture_db.create_schema()
aperture_db.store_embeddings(chunks, embeddings)

# 3. Building the Agentic Workflow


For the agentic flow, we proceed by defining agents using smolAgents in Hugging Face. These agents are responsible for query reformulation, iterative retrievals, and dynamic reranking. All of these are basically an iterative process of rewriting the initial query to get results and prioritizing the results of the iterations for the best result. Here’s how you define the agentic logic for query refinement and retrieval.


In [ ]:
from typing import Dict, Any
from langchain.vectorstores import VectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from smolagents import ToolCallingAgent, LiteLLMModel

class ApertureDBRetriever:
    def __init__(self, aperture_db: ApertureDBVectorStore, k: int = 5):
        self.db = aperture_db
        self.k = k

    def similarity_search(self, query: str, k: int = None) -> List[Dict[str, Any]]:
        """Perform similarity search in ApertureDB"""
        k = k or self.k
        query_vector = model.encode(query).tolist()

        search_query = [
            {
                "FindObject": {
                    "with_vector": {
                        "name": "default_embedding",
                        "vector": query_vector,
                        "k": k
                    },
                    "properties": ["text"],
                    "results": {
                        "list": ["text"]
                    }
                }
            }
        ]

        response, _ = self.db.client.query(search_query)
        if not response or 'FindObject' not in response[0]:
            return []

        results = []
        for i, item in enumerate(response[0]['FindObject']['entities']):
            results.append({
                "content": item['properties']['text'],
                "score": item['vector_distance'],
                "index": i
            })

        return results

class AgenticRetriever:
    def __init__(self, retriever: ApertureDBRetriever):
        self.retriever = retriever

    def __call__(self, query: str) -> str:
        """Retrieve relevant documents and format the output"""
        retrieved_docs = self.retriever.similarity_search(query)

        if not retrieved_docs:
            return "No relevant documents found."

        output = "Retrieved documents:\n"
        for doc in retrieved_docs:
            output += f"\n--- Document {doc['index'] + 1} (score: {doc['score']:.3f}) ---\n"
            output += doc['content'][:500] + ("..." if len(doc['content']) > 500 else "")
            output += "\n"
        return output
aperture_retriever = ApertureDBRetriever(aperture_db)
agentic_retriever = AgenticRetriever(aperture_retriever)


# 4. Integrating with ApertureDB

The last step is to connect the Hugging Face smolAgents defined to the ApertureDB containing the vector embeddings of the dataset RAG is working on. Here’s how you integrate the database with the Hugging Face Agentic AI pipeline for refined query and retrieving results.


In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables (for API keys)
load_dotenv()

def main():
    # Initialize the agent with our retriever tool
    model = LiteLLMModel(model_id="gpt-4-turbo", api_key=os.getenv("OPENAI_API_KEY"))
    agent = ToolCallingAgent(
        tools=[agentic_retriever],
        model=model,
        system_message="You are a helpful research assistant. Use the tools provided to retrieve relevant academic papers."
    )

    queries = [
        "why are graph embeddings used for context preservation",
        "latest research on knowledge graph embeddings",
        "comparison of different graph embedding techniques"
    ]

    for query in queries:
        print(f"\n=== Query: {query} ===")
        response = agent.run(query)
        print("\nResponse:")
        print(response)

if __name__ == "__main__":
    main()